In [127]:
import xml.etree.ElementTree as ET
import string
import os  
import re
import pandas as pd

In [128]:
def clean_text(raw_text):
    '''
    Function to clean the text so that it can easily be used for the classification
    '''
    letters_nums_only = re.sub("[^a-zA-Z0-9_]", " ", str(raw_text)) ## Remove everything except alphabets and numbers
    lower_case = letters_nums_only.lower() # converting the entire text to lower case
    no_blanks_alpha_num_lower = lower_case.strip() #Remove spaces at the beginning and at the end of the string
    return no_blanks_alpha_num_lower

In [136]:
def getcsv(output_file, parent_dir):
    '''
    function to walk through the directories, run clean_text and get the output in a file
    '''

    for root1, dirs, files in os.walk(parent_dir): # To search through the directory in aloop
        for name in files:
            new_path = (os.path.join(root1, name)) # get the path to the directory 
            #print(new_path)          

            abstract_title = (re.findall(r'_(\w+)', name) or None,)[0] #get the abstract name only in the directory and not the full name which is othewise serialized 
            #print( abstract_title)
            if  abstract_title  is not None:
                if abstract_title[0] =='abstract': # finding fulltext.xml file within the directory 
                    #new_path = (os.path.join(root, name)) 
                    
                    tree = ET.parse(new_path) # parse the path 
                    file_full_path = os.path.dirname(new_path) # getting the directory path as a variable
                    file_name = file_full_path.split("\\")[5:6] # getting filename from the full path
                    #print(file_name[0])
                    #print(tree)
                    
                    root = tree.getroot() # navigation to the root of the file
                    #print(root)

                    abstracts = root.findall("./") # searching the fulltext file to get abstract
                    #print(abstracts)

                    for text_ in abstracts: #navigate through every line in the abstract
                        no_blanks_alpha_num_lower  = clean_text(text_.text) # applying the cleantext function
                        #print(output_line)
                        output_line = file_name[0] +  "," + no_blanks_alpha_num_lower  +  "\n" # put output in a string
                        #print(output_line)
                        output_file.write(output_line) # write to a  file

                    
    output_file.close() #close the text file

In [137]:
# Run the functions created 
outfile1 = open('files_with_titles_like_abstract.csv','w')
parent_dir_true  = "C:\\Users\\eless\\ami_12_08_2020\\corpus_950"


getcsv(outfile1, parent_dir_true)



In [138]:
def remove_small_sentences(out_put, in_put):
    '''
    Remove all words like "abstract", "author summary" etc 
    '''
    with open(in_put, "r") as a_file:
        for line in a_file:
            if len(line.split()) >2: # searches for sentences which are longer than 2 words 
                out_put.write(line)

    out_put.close()
    
in_put1 = 'files_with_titles_like_abstract.csv'
#in_put2 = 'not_virus.csv'

out_put1 = open('files_with_abstract_content_only.csv','w')
#out_put2 = open('not_virus_no_junk.csv','w')

remove_small_sentences(out_put1,in_put1)
#remove_small_sentences(out_put2, in_put2)

In [148]:
#add titles like articleID and text to the csv file
import csv 
with open('files_with_abstract_content_only.csv',newline='') as f:
    read_line = csv.reader(f)
    data = [line for line in read_line]
with open('data_with_title.csv','w',newline='') as f:
    write_line = csv.writer(f)
    write_line.writerow(['ArticleId','Text'])
    write_line.writerows(data)

In [149]:
#import the data file as a pandas dataframe
train_data_1 = pd.read_csv("data_with_title.csv")
train_data_1 = train_data_1.iloc[1:] #remove the first row because it appears as data but is acually the title only 
train_data = train_data_1.sample(frac=1).reset_index(drop=True)  #shufffle the dataset

In [151]:
#merge rows that have the same paper ID 
Merged_similar_rows = train_data.groupby('ArticleId', sort=False).Text.unique().agg('  '.join).reset_index()
Merged_similar_rows.head() #visualizing the data table

,ArticleId,Text
0,PMC7188425,this review will briefly examine the clinical ...
1,PMC7294903,all ct scans detected ggos symptomatic patien...
2,PMC7080275,understanding the transition of epidemic to en...
3,PMC7194918,the recent global pandemic created by the coro...
4,PMC7258708,discriminating between patients with microbial...


In [154]:
Merged_similar_rows.to_csv('data_file_final.csv')  ## writing to the CSV file 
